In [1]:
from multiprocessing import Process, Manager, Barrier
import time, logging, os, copy, pathlib, io, sys
import statistics as stats
from PIL import Image
import torchvision.transforms as transforms
import torch

torch.set_num_threads(1)

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s.%(msecs)03d %(levelname)s:\t%(message)s', datefmt='%Y-%m-%d %H:%M:%S')
log = logging.getLogger(__name__)
from shared_memory_dict import SharedMemoryDict
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])

num_p = 1
rep = 1
BATCHSIZE=1
synthetic = False
decoding = True
dir_path = '/home/chanho/ssd2/1024batch/train'

augmentations=[
            transforms.Resize(256),
            transforms.CenterCrop(224),
#             transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(p=1.0),
            transforms.ToTensor(),
            normalize
        ]

resizeAndCrop=[
            transforms.Resize(256),
            transforms.CenterCrop(224)
]

toTensor=[
    transforms.ToTensor(),
]

augmentations_composed=transforms.Compose(augmentations)
augmentations_composed2=transforms.Compose(resizeAndCrop)
augmentations_composed3=transforms.Compose(toTensor)

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')
    
def memloader(byte_data):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    img = Image.open(byte_data)
    return img.convert('RGB')
    
def aug(imgd,result,barrier,num,preloading,num_p,rep):
    import psutil
    p = psutil.Process()
    p.cpu_affinity([num])
    
    for j in range(rep):
        arr = []
        
        barrier.wait()
        log.debug(f"Start Process{num}")
        start=time.perf_counter()
    
        for i in range(BATCHSIZE):
            if preloading:
                print(f"before decoding img size: get_size of {imgd[num+num_p*i].getbuffer().nbytes} bytes")
                img = memloader(imgd[num+num_p*i])
                print(f"after decoding img size: get_size of {sys.getsizeof(img.tobytes())} bytes")
            else:
                img = imgd[num+num_p*i]
                
            sample3 = augmentations_composed3(img)
            print(f"no crop resize sample size: get_size of {sample3.element_size() * sample3.nelement()} bytes")
            print(f"no crop resize sample element size: {sample3.element_size()}, sample nelement: {sample3.nelement()}")
            print(f"Type: {sample3.dtype}, {sample3.shape}")
            sample = augmentations_composed(img)
            sample2 = augmentations_composed2(img)
            print(f"resized and crop size: get_size of {sys.getsizeof(sample2.tobytes())} bytes")
            
            
            print(f"sample size: get_size of {sample.element_size() * sample.nelement()} bytes")
            print(f"sample element size: {sample.element_size()}, sample nelement: {sample.nelement()}")
            print(f"Type: {sample.dtype}, {sample.shape}")
#             arr.append(sample)
        end=time.perf_counter()                            
    
        elapsed_time=end-start
        log.debug(f"End Process{num} Access time: {elapsed_time}")
        result.append(elapsed_time)
        
    return arr


manager1 = Manager()
manager2 = Manager()
manager3 = Manager()
manager4 = Manager()

# d = [{},{},{},{}]
# d = [manager1.dict(),manager1.dict(),manager1.dict(),manager1.dict()]
# d = [manager1.dict(),manager2.dict(),manager3.dict(),manager4.dict()]
d = [manager1.dict(),manager2.dict(),manager3.dict(),manager4.dict()]
results = [manager1.list(),manager2.list(),manager3.list(),manager4.list()]

dataset_size = 30 # MB
size = dataset_size * 1024 * 1024

for i in range(num_p*256):
    if synthetic:
        d[i % num_p][i] = torch.rand((3, 224, 224))
    elif decoding:
        path =pathlib.Path(f'/home/chanho/ssd2/1024batch/train/n02364673/{i}.JPEG')
#         print(f"path size: {os.stat(path).st_size}")
        with open(path, 'rb') as f:
            img_byte = f.read()
            img_byte_mem = io.BytesIO(img_byte)
#         print(f"img_byte_mem size: get_size of {sys.getsizeof(img_byte_mem.tell())} bytes")
        d[i % num_p][i] = img_byte_mem
    else:
        path =pathlib.Path(f'/home/chanho/ssd2/1024batch/train/n02364673/{i}.JPEG')
        d[i % num_p][i] = pil_loader(path)
#         print(f"img_byte_mem size: get_size of {sys.getsizeof(d[i % num_p][i].tell())} bytes")
        
        
    
#         smd[i % 4][i] = f"test{i}"
barrier=Barrier(num_p)
ps = []
for i in range(num_p):
#         p = Process(target=shdf, args=(smd[i % 4],barrier,i))
    p=Process(target=aug, args=(d[i % num_p],results[i % num_p],barrier,i,decoding, num_p, rep))
    ps.append(p)


for p in ps:
    p.start()

for p in ps:
    p.join()
    
# for single_d in d:
#     print(d)

2021-10-26 19:27:17.434 DEBUG:	Start Process0


before decoding img size: get_size of 108063 bytes
after decoding img size: get_size of 562533 bytes
no crop resize sample size: get_size of 2250000 bytes
no crop resize sample element size: 4, sample nelement: 562500
Type: torch.float32, torch.Size([3, 375, 500])
resized and crop size: get_size of 150561 bytes
sample size: get_size of 602112 bytes
sample element size: 4, sample nelement: 150528
Type: torch.float32, torch.Size([3, 224, 224])


2021-10-26 19:27:17.488 DEBUG:	End Process0 Access time: 0.031310392543673515


In [2]:
def stat(data):
#     print("max: ", stats.fmean(data))  
#     print("min: ",stats.min())  
    print("mean: ",stats.fmean(data))  
    print("median: ",stats.median(data))  
    print("std: ",stats.stdev(data))  
#     print("10large: ",data.nlargest(10))
#     print("10small: ",data.nsmallest(10))
    print("quantile",stats.quantiles(data, n=6))
    
for i in range(num_p) :
    print(f"\nProcess{i}")
    stat(results[i])
    


Process0
mean:  0.031310392543673515
median:  0.031310392543673515


StatisticsError: variance requires at least two data points

In [ ]:
from PIL import features
features.check_feature("libjpeg_turbo")

In [ ]:
# from multiprocessing import Process, Manager, Barrier
# import time, logging, os, copy, pathlib
# from PIL import Image
# import torchvision.transforms as transforms
# logging.basicConfig(level=logging.DEBUG, format='%(asctime)s.%(msecs)03d %(levelname)s:\t%(message)s', datefmt='%Y-%m-%d %H:%M:%S')
# log = logging.getLogger(__name__)
# from shared_memory_dict import SharedMemoryDict

# def pil_loader(path):
#     # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
#     with open(path, 'rb') as f:
#         img = Image.open(f)
#         return img.convert('RGB')

# def f(d,barrier,num):
#     barrier.wait()
#     log.debug(f"Start Process{num}")
#     start=time.perf_counter()
#     for i in range(256):
#         item=d[num+4*i]
#     end=time.perf_counter()
#     log.debug(f"End Process{num} Access time: {end-start}")
#     return

# num_p = 4

# manager1 = Manager()
# manager2 = Manager()
# manager3 = Manager()
# manager4 = Manager()

# d = [manager1.dict(),manager2.dict(),manager3.dict(),manager4.dict()]
# for i in range(num_p*256):
#     path =pathlib.Path(f'/home/chanho/ssd2/1024batch/train/n02364673/{i}.JPEG')
#     d[i % 4][i] = pil_loader(path)
# barrier=Barrier(num_p)
# ps = []
# for i in range(num_p):
#     p = Process(target=f, args=(d[i % 4],barrier,i))
#     ps.append(p)

# for p in ps:
#     p.start()

# for p in ps:
#     p.join()

In [ ]:
# def f(d,barrier,num):
#     barrier.wait()
#     log.debug(f"Start Process{num}")
#     start=time.perf_counter()
#     for i in range(256):
#         item=d[num+4*i]
#     end=time.perf_counter()
#     log.debug(f"End Process{num} Access time: {end-start}")
#     return

# def pil_loader(path):
#     # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
#     with open(path, 'rb') as f:
#         img = Image.open(f)
#         return img.convert('RGB')

# num_p = 4
# manager1 = Manager()
# manage_dict=manager1.dict()
# d = [manage_dict,manage_dict,manage_dict,manage_dict]
# for i in range(num_p*256):
#     path =pathlib.Path(f'/home/chanho/ssd2/1024batch/train/n02364673/{i}.JPEG')
#     d[i % 4][i] = pil_loader(path)
# barrier=Barrier(num_p)
# ps = []
# for i in range(num_p):
#     p=Process(target=f, args=(d[i % 4],barrier,i))
#     ps.append(p)

# for p in ps:
#     p.start()

# for p in ps:
#     p.join()

In [ ]:
# num_p = 4

# d = [{},{},{},{}]
# for i in range(num_p*256):
#     path =pathlib.Path(f'/home/chanho/ssd2/1024batch/train/n02364673/{i}.JPEG')
#     d[i % 4][i] = pil_loader(path)
# barrier=Barrier(num_p)
# ps = []
# for i in range(num_p):
#     p=Process(target=f, args=(d[i % 4],barrier,i))
#     ps.append(p)

# for p in ps:
#     p.start()

# for p in ps:
#     p.join()